<a href="https://colab.research.google.com/github/valenderval/MLA_JFV/blob/main/FinalProject_Classification_JFV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Final Project - Classification
Jacob Vallery <br>CS430-ON <br>Machine Learning in the Cloud